### Here we'll use data from Lending Club (2015) to predict the state of a loan given some information about it. 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

%matplotlib inline

In [3]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


## The Blind Approach

In [ ]:
#do not run, kernel will die, not executed
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)

cross_val_score(rfc, X, Y, cv=5)

### Data Cleaning:

In [5]:
#Lets look at all our categorical variables and see how many distinct counts there are...
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense.

In [6]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

Wonder what was causing the dtype error on the id column, which should have all been integers? Let's look at the end of the file.

In [7]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806.0,39222577.0,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262.0,38982659.0,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [9]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,68009401.0,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,...,0,0,0,0,0,1,0,0,0,0
1,68354783.0,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,...,0,0,0,0,0,1,0,0,0,0
2,68466916.0,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,...,0,0,0,0,0,1,0,0,0,0
3,68466961.0,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,...,0,0,0,0,0,1,0,0,0,0
4,68495092.0,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,...,0,0,0,0,0,1,0,0,0,0
5,68506798.0,73396623.0,23000.0,23000.0,23000.0,8.49,471.77,64000.0,18.28,0.0,...,0,0,0,0,0,1,0,0,0,0
6,68566886.0,73456723.0,29900.0,29900.0,29900.0,12.88,678.49,65000.0,21.77,0.0,...,0,0,0,0,0,1,0,0,0,0
7,68577849.0,73467703.0,18000.0,18000.0,18000.0,11.99,400.31,112000.0,8.68,0.0,...,0,0,0,0,0,1,0,0,0,0
8,66310712.0,71035433.0,35000.0,35000.0,35000.0,14.85,829.90,110000.0,17.06,0.0,...,0,0,0,0,0,1,0,0,0,0
9,68476807.0,73366655.0,10400.0,10400.0,10400.0,22.45,289.91,104433.0,25.37,1.0,...,0,0,0,0,0,1,0,0,0,0


It finally works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

### Second Attempt:

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

In [10]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anacon

array([0.98029019, 0.98038517, 0.98140628, 0.98190497, 0.97031584,
       0.97499406, 0.95955734, 0.98045548, 0.97995583, 0.98064409])

- The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.
- That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

- There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

### DRILL: Third Attempt
- So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

- You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

- Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [11]:
y2015.isnull().sum()

id                                     0
member_id                              0
loan_amnt                              0
funded_amnt                            0
funded_amnt_inv                        0
term                                   0
int_rate                               0
installment                            0
grade                                  0
emp_length                         23817
home_ownership                         0
annual_inc                             0
verification_status                    0
issue_d                                0
loan_status                            0
pymnt_plan                             0
purpose                                0
title                                132
dti                                    0
delinq_2yrs                            0
inq_last_6mths                         0
mths_since_last_delinq            203962
mths_since_last_record            346680
open_acc                               0
pub_rec         

In [12]:
y2015.iloc[:, 50:75].isnull().sum()

tot_cur_bal                      0
open_acc_6m                 399723
open_il_6m                  399723
open_il_12m                 399723
open_il_24m                 399723
mths_since_rcnt_il          400285
total_bal_il                399723
il_util                     402478
open_rv_12m                 399723
open_rv_24m                 399723
max_bal_bc                  399723
all_util                    399723
total_rev_hi_lim                 0
inq_fi                      399723
total_cu_tl                 399723
inq_last_12m                399723
acc_open_past_24mths             0
avg_cur_bal                      0
bc_open_to_buy                3963
bc_util                       4227
chargeoff_within_12_mths         0
delinq_amnt                      0
mo_sin_old_il_acct           12254
mo_sin_old_rev_tl_op             0
mo_sin_rcnt_rev_tl_op            0
dtype: int64

- dropping all the features with missing values is higher

In [13]:
y2015.drop(['mths_since_last_delinq', 'mths_since_last_record', 'mths_since_recent_bc_dlq', 
           'mths_since_recent_revol_delinq', 'emp_length', 'title', 'num_tl_120dpd_2m', 'percent_bc_gt_75', 
           'next_pymnt_d', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 
           'open_acc_6m', 'inq_last_12m', 'total_cu_tl', 'inq_fi', 'all_util', 'max_bal_bc', 'open_rv_24m', 
           'open_rv_12m', 'il_util', 'total_bal_il', 'mths_since_rcnt_il', 'open_il_24m', 'open_il_12m', 
           'open_il_6m', 'open_acc_6m'], axis = 1, inplace= True)


In [14]:
y2015.shape

(421095, 76)

In [15]:
pd.get_dummies(y2015).shape

(421095, 169)

In [16]:
y2015_t = pd.get_dummies(y2015)

y2015_t.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,last_credit_pull_d_May-2015,last_credit_pull_d_May-2016,last_credit_pull_d_Nov-2015,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT
0,68009401.0,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,...,0,0,0,0,0,0,0,0,1,0
1,68354783.0,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,...,0,0,0,0,0,0,0,0,1,0
2,68466916.0,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,...,0,0,0,0,0,0,0,0,1,0
3,68466961.0,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,...,0,0,0,0,0,0,0,0,1,0
4,68495092.0,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,...,0,0,0,0,0,0,0,0,1,0


### Finding correlation between all features:

In [17]:
y2015_t.iloc[:, :50].corr()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl
id,1.000000,0.997540,-0.008288,-0.008288,-0.008554,-0.053402,-0.012919,0.015055,0.005692,0.002300,...,-0.005305,-0.013036,0.004085,-0.009252,-0.006172,-0.006015,-0.014951,0.008984,-0.028205,0.000665
member_id,0.997540,1.000000,-0.008620,-0.008620,-0.008885,-0.053447,-0.013274,0.014958,0.005806,0.002261,...,-0.005469,-0.013261,0.004029,-0.009397,-0.006180,-0.005916,-0.014976,0.008889,-0.028132,0.000371
loan_amnt,-0.008288,-0.008620,1.000000,1.000000,0.999994,0.140572,0.941205,0.305734,0.006103,-0.010424,...,0.041686,0.231574,0.051817,0.008587,-0.057356,0.212990,0.164565,0.222171,0.203509,0.074857
funded_amnt,-0.008288,-0.008620,1.000000,1.000000,0.999994,0.140572,0.941205,0.305734,0.006103,-0.010424,...,0.041686,0.231574,0.051817,0.008587,-0.057356,0.212990,0.164565,0.222171,0.203509,0.074857
funded_amnt_inv,-0.008554,-0.008885,0.999994,0.999994,1.000000,0.140209,0.941187,0.305803,0.006044,-0.010448,...,0.041750,0.231630,0.051856,0.008647,-0.057389,0.212987,0.164525,0.222211,0.203576,0.074842
int_rate,-0.053402,-0.053447,0.140572,0.140572,0.140209,1.000000,0.124426,-0.090399,0.077932,0.043710,...,-0.150179,-0.076131,-0.096828,-0.184545,0.044632,-0.018758,0.066422,-0.083299,-0.115968,0.033268
installment,-0.012919,-0.013274,0.941205,0.941205,0.941187,0.124426,1.000000,0.297393,0.001893,-0.002129,...,0.027120,0.202128,0.033057,-0.011824,-0.045646,0.202271,0.161092,0.208238,0.186971,0.056563
annual_inc,0.015055,0.014958,0.305734,0.305734,0.305803,-0.090399,0.297393,1.000000,-0.068237,0.037910,...,-0.021290,0.209046,0.025358,-0.038174,0.016400,0.106892,0.074204,0.120446,0.125379,0.084127
dti,0.005692,0.005806,0.006103,0.006103,0.006044,0.077932,0.001893,-0.068237,1.000000,-0.005868,...,-0.037806,-0.020044,0.001114,-0.001369,-0.022060,0.062764,0.094225,0.040617,0.028171,0.081730
delinq_2yrs,0.002300,0.002261,-0.010424,-0.010424,-0.010448,0.043710,-0.002129,0.037910,-0.005868,1.000000,...,0.016409,0.072891,0.054750,-0.031239,0.202065,-0.051981,-0.013438,-0.034428,0.028650,0.082187


In [18]:
y2015_t.iloc[:, 50:100].corr()

,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,...,issue_d_Oct-2015,issue_d_Sep-2015,loan_status_Charged Off,loan_status_Current,loan_status_Default,loan_status_Fully Paid,loan_status_In Grace Period,loan_status_Late (16-30 days),loan_status_Late (31-120 days),pymnt_plan_n
num_op_rev_tl,1.000000,0.781827,0.811682,0.828263,0.011889,-0.030011,0.344745,0.147413,-0.002197,-0.003370,...,0.000114,0.003899,0.010101,-0.002923,0.002465,-0.006038,-0.004023,0.002635,0.009238,NaN
num_rev_accts,0.781827,1.000000,0.580886,0.643036,0.019635,0.034136,0.306660,0.030236,0.082573,-0.020558,...,-0.001434,0.000109,0.002243,-0.033974,0.001594,0.039406,-0.006741,-0.000970,-0.000681,NaN
num_rev_tl_bal_gt_0,0.811682,0.580886,1.000000,0.673746,0.001427,-0.028787,0.255468,0.117217,-0.032856,0.004761,...,-0.004809,0.001859,0.017313,0.035653,0.004359,-0.061738,0.008881,0.008074,0.016717,NaN
num_sats,0.828263,0.643036,0.673746,1.000000,0.007047,-0.000404,0.358021,0.131610,-0.040522,-0.010459,...,0.000005,0.004343,0.010211,-0.004537,0.003719,-0.006783,0.002361,0.005344,0.010230,NaN
num_tl_30dpd,0.011889,0.019635,0.001427,0.007047,1.000000,0.006916,-0.007331,-0.040701,-0.014003,0.003628,...,0.003219,-0.003159,0.002211,-0.000952,-0.001784,-0.000055,0.002614,-0.001376,-0.001279,NaN
num_tl_90g_dpd_24m,-0.030011,0.034136,-0.028787,-0.000404,0.006916,1.000000,0.012494,-0.275400,0.014490,0.000522,...,0.000969,-0.003796,0.004150,-0.003996,-0.000949,-0.001855,0.005930,0.003191,0.005321,NaN
num_tl_op_past_12m,0.344745,0.306660,0.255468,0.358021,-0.007331,0.012494,1.000000,-0.003231,0.126741,0.018948,...,-0.009196,-0.003308,0.082550,-0.126972,0.012649,0.071918,0.013507,0.011912,0.041698,NaN
pct_tl_nvr_dlq,0.147413,0.030236,0.117217,0.131610,-0.040701,-0.275400,-0.003231,1.000000,-0.006280,-0.039201,...,-0.001641,0.004055,0.007950,-0.010820,-0.005022,0.015095,-0.012517,-0.005245,-0.008680,NaN
pub_rec_bankruptcies,-0.002197,0.082573,-0.032856,-0.040522,-0.014003,0.014490,0.126741,-0.006280,1.000000,0.035171,...,-0.005276,0.000644,0.016107,-0.040475,0.005380,0.032111,0.000038,0.000786,0.009457,NaN
tax_liens,-0.003370,-0.020558,0.004761,-0.010459,0.003628,0.000522,0.018948,-0.039201,0.035171,1.000000,...,-0.001445,-0.001634,0.001461,0.013446,0.001183,-0.017280,0.002765,0.001290,-0.000141,NaN


- looking above, I couldn't see any correlation between loan status and any other features, so I will not use any features 50 -100.

In [19]:
y2015_t.iloc[:, 92: 140].corr()

,loan_status_Charged Off,loan_status_Current,loan_status_Default,loan_status_Fully Paid,loan_status_In Grace Period,loan_status_Late (16-30 days),loan_status_Late (31-120 days),pymnt_plan_n,purpose_car,purpose_credit_card,...,last_pymnt_d_Jun-2016,last_pymnt_d_Mar-2015,last_pymnt_d_Mar-2016,last_pymnt_d_May-2015,last_pymnt_d_May-2016,last_pymnt_d_Nov-2015,last_pymnt_d_Nov-2016,last_pymnt_d_Oct-2015,last_pymnt_d_Oct-2016,last_pymnt_d_Sep-2015
loan_status_Charged Off,1.000000,-0.400083,-0.011874,-0.140234,-0.027779,-0.018311,-0.041475,NaN,-0.004053,-0.042153,...,0.170252,0.013430,0.131679,0.034246,0.156156,0.083127,-0.031745,0.099567,-0.022168,0.082330
loan_status_Current,-0.400083,1.000000,-0.063811,-0.753602,-0.149283,-0.098402,-0.222884,NaN,-0.001789,0.066653,...,-0.205375,-0.058666,-0.208300,-0.086191,-0.195865,-0.150898,-0.215631,-0.155042,-0.216986,-0.136158
loan_status_Default,-0.011874,-0.063811,1.000000,-0.022367,-0.004431,-0.002921,-0.006615,NaN,-0.002754,-0.003938,...,-0.006097,-0.001741,-0.005004,-0.002558,-0.004147,-0.003405,-0.006400,-0.004602,-0.006440,-0.004041
loan_status_Fully Paid,-0.140234,-0.753602,-0.022367,1.000000,-0.052326,-0.034491,-0.078124,NaN,0.005868,-0.039608,...,0.142312,0.062639,0.169819,0.082963,0.139422,0.130659,0.166468,0.125521,0.165836,0.113425
loan_status_In Grace Period,-0.027779,-0.149283,-0.004431,-0.052326,1.000000,-0.006832,-0.015476,NaN,-0.002493,-0.008837,...,-0.014263,-0.004073,-0.014463,-0.005985,-0.013600,-0.010477,-0.014972,-0.010765,-0.015066,-0.009454
loan_status_Late (16-30 days),-0.018311,-0.098402,-0.002921,-0.034491,-0.006832,1.000000,-0.010201,NaN,-0.002966,-0.007419,...,-0.009401,-0.002685,-0.009534,-0.003945,-0.008964,-0.006906,-0.003696,-0.007096,-0.009931,-0.006232
loan_status_Late (31-120 days),-0.041475,-0.222884,-0.006615,-0.078124,-0.015476,-0.010201,1.000000,NaN,0.000305,-0.017911,...,-0.021294,-0.006082,-0.021594,-0.008935,-0.020305,-0.015643,0.288003,-0.016073,0.280488,-0.014115
pymnt_plan_n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
purpose_car,-0.004053,-0.001789,-0.002754,0.005868,-0.002493,-0.002966,0.000305,NaN,1.000000,-0.051514,...,-0.001858,0.004909,0.003480,0.001824,-0.000953,-0.000341,0.000115,0.004192,0.001039,0.001247
purpose_credit_card,-0.042153,0.066653,-0.003938,-0.039608,-0.008837,-0.007419,-0.017911,NaN,-0.051514,1.000000,...,-0.010517,-0.007778,-0.016209,-0.009491,-0.013791,-0.017047,-0.008730,-0.018290,-0.005602,-0.015847


I can see that there might be some correlation between last payment, so I will use those but not others.

In [20]:
y2015_t.iloc[:, 92: ].corr()

,loan_status_Charged Off,loan_status_Current,loan_status_Default,loan_status_Fully Paid,loan_status_In Grace Period,loan_status_Late (16-30 days),loan_status_Late (31-120 days),pymnt_plan_n,purpose_car,purpose_credit_card,...,last_credit_pull_d_May-2015,last_credit_pull_d_May-2016,last_credit_pull_d_Nov-2015,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT
loan_status_Charged Off,1.000000,-0.400083,-0.011874,-0.140234,-0.027779,-0.018311,-0.041475,NaN,-0.004053,-0.042153,...,-0.002618,-0.016295,-0.008034,0.062597,-0.005538,0.350309,-0.002971,-0.020383,0.004139,-0.004139
loan_status_Current,-0.400083,1.000000,-0.063811,-0.753602,-0.149283,-0.098402,-0.222884,NaN,-0.001789,0.066653,...,-0.038990,-0.111738,-0.074012,-0.177317,-0.070608,-0.297041,-0.067725,-0.109058,-0.005455,0.005455
loan_status_Default,-0.011874,-0.063811,1.000000,-0.022367,-0.004431,-0.002921,-0.006615,NaN,-0.002754,-0.003938,...,-0.001185,-0.003373,-0.002300,-0.005405,-0.002172,-0.008217,-0.002074,-0.003251,-0.001624,0.001624
loan_status_Fully Paid,-0.140234,-0.753602,-0.022367,1.000000,-0.052326,-0.034491,-0.078124,NaN,0.005868,-0.039608,...,0.048903,0.145285,0.094634,0.174036,0.089115,0.140089,0.083870,0.144646,0.005163,-0.005163
loan_status_In Grace Period,-0.027779,-0.149283,-0.004431,-0.052326,1.000000,-0.006832,-0.015476,NaN,-0.002493,-0.008837,...,-0.002772,-0.007586,-0.004934,-0.011681,-0.005081,-0.020676,-0.004356,-0.007606,-0.002544,0.002544
loan_status_Late (16-30 days),-0.018311,-0.098402,-0.002921,-0.034491,-0.006832,1.000000,-0.010201,NaN,-0.002966,-0.007419,...,-0.001827,-0.005202,-0.003547,-0.006591,-0.003349,-0.013227,-0.003198,-0.005013,0.000297,-0.000297
loan_status_Late (31-120 days),-0.041475,-0.222884,-0.006615,-0.078124,-0.015476,-0.010201,1.000000,NaN,0.000305,-0.017911,...,-0.004139,-0.011160,-0.007731,-0.015220,-0.007585,-0.029089,-0.007243,-0.010925,-0.002047,0.002047
pymnt_plan_n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
purpose_car,-0.004053,-0.001789,-0.002754,0.005868,-0.002493,-0.002966,0.000305,NaN,1.000000,-0.051514,...,0.000417,0.000784,0.001170,-0.002289,0.004430,-0.002690,0.001187,0.000624,0.002420,-0.002420
purpose_credit_card,-0.042153,0.066653,-0.003938,-0.039608,-0.008837,-0.007419,-0.017911,NaN,-0.051514,1.000000,...,-0.002565,-0.004772,-0.003807,-0.005841,-0.004724,-0.024008,-0.005262,-0.007626,0.001402,-0.001402


In [21]:
y2015_t.iloc[:, : 100].corr()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,issue_d_Oct-2015,issue_d_Sep-2015,loan_status_Charged Off,loan_status_Current,loan_status_Default,loan_status_Fully Paid,loan_status_In Grace Period,loan_status_Late (16-30 days),loan_status_Late (31-120 days),pymnt_plan_n
id,1.000000,0.997540,-0.008288,-0.008288,-0.008554,-0.053402,-0.012919,0.015055,0.005692,0.002300,...,0.306377,0.167767,-0.083406,0.146429,0.004373,-0.119724,-0.000731,0.005020,0.008371,NaN
member_id,0.997540,1.000000,-0.008620,-0.008620,-0.008885,-0.053447,-0.013274,0.014958,0.005806,0.002261,...,0.304707,0.164344,-0.083382,0.146156,0.004429,-0.119457,-0.000748,0.005096,0.008412,NaN
loan_amnt,-0.008288,-0.008620,1.000000,1.000000,0.999994,0.140572,0.941205,0.305734,0.006103,-0.010424,...,-0.002539,0.015131,0.011895,0.015382,0.001988,-0.033211,0.011779,0.009891,0.009339,NaN
funded_amnt,-0.008288,-0.008620,1.000000,1.000000,0.999994,0.140572,0.941205,0.305734,0.006103,-0.010424,...,-0.002539,0.015131,0.011895,0.015382,0.001988,-0.033211,0.011779,0.009891,0.009339,NaN
funded_amnt_inv,-0.008554,-0.008885,0.999994,0.999994,1.000000,0.140209,0.941187,0.305803,0.006044,-0.010448,...,-0.002523,0.015019,0.011844,0.015401,0.001965,-0.033178,0.011769,0.009872,0.009298,NaN
int_rate,-0.053402,-0.053447,0.140572,0.140572,0.140209,1.000000,0.124426,-0.090399,0.077932,0.043710,...,-0.026567,0.003717,0.191282,-0.144180,0.023909,-0.003648,0.043713,0.034146,0.082766,NaN
installment,-0.012919,-0.013274,0.941205,0.941205,0.941187,0.124426,1.000000,0.297393,0.001893,-0.002129,...,-0.003417,0.013160,0.021777,-0.014418,0.002959,-0.007999,0.015357,0.011804,0.013250,NaN
annual_inc,0.015055,0.014958,0.305734,0.305734,0.305803,-0.090399,0.297393,1.000000,-0.068237,0.037910,...,0.005623,0.007453,-0.026646,0.003351,-0.001643,0.017387,-0.000508,-0.001678,-0.010951,NaN
dti,0.005692,0.005806,0.006103,0.006103,0.006044,0.077932,0.001893,-0.068237,1.000000,-0.005868,...,-0.001188,-0.000510,0.027532,0.004559,0.003209,-0.028381,0.005063,0.004066,0.010120,NaN
delinq_2yrs,0.002300,0.002261,-0.010424,-0.010424,-0.010448,0.043710,-0.002129,0.037910,-0.005868,1.000000,...,0.001736,-0.005366,0.006135,0.000854,0.000028,-0.013399,0.012607,0.006480,0.012031,NaN


In [22]:
y2015_t.iloc[:, 28: 99].corr()

,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,...,issue_d_Nov-2015,issue_d_Oct-2015,issue_d_Sep-2015,loan_status_Charged Off,loan_status_Current,loan_status_Default,loan_status_Fully Paid,loan_status_In Grace Period,loan_status_Late (16-30 days),loan_status_Late (31-120 days)
tot_coll_amt,1.000000,-0.007708,-0.026552,0.019328,-0.005693,-0.015698,-0.037458,0.013216,0.002142,0.018129,...,0.001121,0.000071,-0.002139,-0.002797,-0.000171,-0.000348,0.001692,-0.000561,0.003098,-0.000224
tot_cur_bal,-0.007708,1.000000,0.410690,0.070250,0.834840,0.163424,0.045060,0.006576,0.017337,0.170101,...,0.002993,0.009167,0.007581,-0.043356,-0.002189,-0.008145,0.039465,-0.003728,-0.002539,-0.020978
total_rev_hi_lim,-0.026552,0.410690,1.000000,0.067779,0.225868,0.562399,-0.151831,-0.013072,-0.000032,0.086135,...,0.008066,0.011103,0.010010,-0.036581,0.025934,-0.006605,0.006790,-0.014248,-0.007676,-0.022262
acc_open_past_24mths,0.019328,0.070250,0.067779,1.000000,-0.102708,0.085986,-0.168829,0.007793,-0.000521,-0.001819,...,-0.000012,-0.009075,0.003403,0.088433,-0.132478,0.014486,0.071736,0.015246,0.016068,0.045804
avg_cur_bal,-0.005693,0.834840,0.225868,-0.102708,1.000000,0.042099,0.076760,0.007553,0.016753,0.116849,...,0.002479,0.008026,0.005468,-0.045598,-0.004915,-0.009216,0.045476,-0.004719,-0.004113,-0.023359
bc_open_to_buy,-0.015698,0.163424,0.562399,0.085986,0.042099,1.000000,-0.539107,-0.009835,0.001050,0.032111,...,0.009205,0.009371,0.006066,-0.048737,0.005364,-0.007332,0.044218,-0.025689,-0.013729,-0.028906
bc_util,-0.037458,0.045060,-0.151831,-0.168829,0.076760,-0.539107,1.000000,-0.016619,-0.005817,0.042391,...,-0.010963,-0.005342,-0.002538,0.027208,0.048001,0.004374,-0.087121,0.025631,0.012457,0.017356
chargeoff_within_12_mths,0.013216,0.006576,-0.013072,0.007793,0.007553,-0.009835,-0.016619,1.000000,0.007182,0.019460,...,0.003057,0.001412,-0.000439,0.003512,-0.001787,-0.000605,0.000455,0.003420,-0.001420,-0.003155
delinq_amnt,0.002142,0.017337,-0.000032,-0.000521,0.016753,0.001050,-0.005817,0.007182,1.000000,0.003759,...,-0.001245,0.003642,0.001301,0.001832,-0.001052,-0.000763,0.000707,-0.001243,-0.000860,-0.000319
mo_sin_old_il_acct,0.018129,0.170101,0.086135,-0.001819,0.116849,0.032111,0.042391,0.019460,0.003759,1.000000,...,0.001485,0.000365,-0.001620,-0.030098,0.031891,-0.003234,-0.013532,-0.002510,-0.001258,-0.008203


Only features that are correlated to Loan status:

In [23]:
y2015_t1 = y2015_t[['loan_status_Charged Off', 'loan_status_Current', 'loan_status_Default', 'loan_status_Fully Paid', 
                    'loan_status_In Grace Period', 'loan_status_Late (16-30 days)', 'loan_status_Late (31-120 days)', 
                    'id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 
                    'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 
                    'revol_bal', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 
                    'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 
                    'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq'
                    
                    ]]

In [24]:
y2015_t1.shape

(421095, 35)

In [25]:
#checking correlation for further selection
y2015_t1.corr() 

,loan_status_Charged Off,loan_status_Current,loan_status_Default,loan_status_Fully Paid,loan_status_In Grace Period,loan_status_Late (16-30 days),loan_status_Late (31-120 days),id,member_id,loan_amnt,...,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq
loan_status_Charged Off,1.000000,-0.400083,-0.011874,-0.140234,-0.027779,-0.018311,-0.041475,-0.083406,-0.083382,0.011895,...,-0.176329,-0.204028,-0.069750,0.103992,0.462945,0.464829,-0.104859,0.008635,NaN,0.005234
loan_status_Current,-0.400083,1.000000,-0.063811,-0.753602,-0.149283,-0.098402,-0.222884,0.146429,0.146156,0.015382,...,-0.320022,-0.389478,0.180020,-0.073964,-0.185216,-0.185970,-0.569426,0.001837,NaN,-0.001495
loan_status_Default,-0.011874,-0.063811,1.000000,-0.022367,-0.004431,-0.002921,-0.006615,0.004373,0.004429,0.001988,...,-0.020246,-0.023008,0.004247,0.009002,-0.005497,-0.005520,-0.015336,-0.001080,NaN,-0.002431
loan_status_Fully Paid,-0.140234,-0.753602,-0.022367,1.000000,-0.052326,-0.034491,-0.078124,-0.119724,-0.119457,-0.033211,...,0.503716,0.611635,-0.195000,-0.029847,-0.064921,-0.065185,0.753773,-0.010047,NaN,-0.001725
loan_status_In Grace Period,-0.027779,-0.149283,-0.004431,-0.052326,1.000000,-0.006832,-0.015476,-0.000731,-0.000748,0.011779,...,-0.015425,-0.028790,0.044390,0.040559,-0.012860,-0.012913,-0.038199,0.002628,NaN,0.001555
loan_status_Late (16-30 days),-0.018311,-0.098402,-0.002921,-0.034491,-0.006832,1.000000,-0.010201,0.005020,0.005096,0.009891,...,-0.011910,-0.021241,0.030497,0.038381,-0.008477,-0.008512,-0.024854,0.000629,NaN,-0.001024
loan_status_Late (31-120 days),-0.041475,-0.222884,-0.006615,-0.078124,-0.015476,-0.010201,1.000000,0.008371,0.008412,0.009339,...,-0.052611,-0.068905,0.043749,0.088261,-0.019201,-0.019279,-0.057809,0.005228,NaN,0.000576
id,-0.083406,0.146429,0.004373,-0.119724,-0.000731,0.005020,0.008371,1.000000,0.997540,-0.008288,...,-0.220078,-0.188353,-0.165885,-0.015779,-0.070027,-0.070594,-0.068844,0.011953,NaN,-0.004834
member_id,-0.083382,0.146156,0.004429,-0.119457,-0.000748,0.005096,0.008412,0.997540,1.000000,-0.008620,...,-0.220139,-0.188443,-0.165861,-0.015748,-0.069799,-0.070372,-0.068974,0.011891,NaN,-0.004732
loan_amnt,0.011895,0.015382,0.001988,-0.033211,0.011779,0.009891,0.009339,-0.008288,-0.008620,1.000000,...,0.706157,0.556266,0.736585,0.051888,0.077870,0.079190,0.259864,-0.027454,NaN,0.000645


- out_prncp, out_prncp_inv, total_pymnt, total_pymnt_inv, total_rec_prncp, recoveries, collection_recovery_fee, last_pymnt_amnt
- highly correlated: recoveries and collection_recovery_fee, total_pymnt, last_pymnt_amnt with total_pymnt_inv, total_rec_prncp, total_pymnt.

#### First datasets with all correlated features:

In [26]:
y2015_ls = y2015_t1[["out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp", 
                     "recoveries", "collection_recovery_fee", "last_pymnt_amnt"
                    ]]

#### Second datasets only the features that are not multicollinearity:

In [27]:
y2015_ls1 = y2015_t1[["out_prncp", "out_prncp_inv", "collection_recovery_fee", "last_pymnt_amnt"
                    ]]

In [28]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015_ls #using first datasets
Y = y2015['loan_status']
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anacon

array([0.79898364, 0.9596305 , 0.94414761, 0.9526252 , 0.94939444,
       0.942199  , 0.91077917, 0.92825762, 0.94447479, 0.95955446])

In [29]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015_ls1 #Using second datasets
Y = y2015['loan_status']
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anacon

array([0.87259861, 0.93524257, 0.91384674, 0.94284154, 0.94191403,
       0.93595346, 0.91453134, 0.92823387, 0.94271736, 0.94243101])

#### Third datasets using correlated features without outstanding prncp related and total payment related:

In [31]:
y2015_ls2 = y2015_t[["total_rec_prncp", "recoveries", "collection_recovery_fee", "last_credit_pull_d_Jan-2017"
                     ]]

In [32]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015_ls2 #Using third datasets
Y = y2015['loan_status']
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Top-Secret/anacon

array([0.83472252, 0.86552207, 0.84493363, 0.89708152, 0.88793636,
       0.88242698, 0.85623026, 0.86964782, 0.88213361, 0.84781266])

- Accuracy went down.